## Este notebook funciona con los datos v6 (7 variables)


In [1]:
import sys
import requests
import inspect
import pandas as pd
import random
import csv

from skmultiflow.trees.hoeffding_adaptive_tree import HoeffdingAdaptiveTreeClassifier
from skmultiflow.data.file_stream import FileStream
from skmultiflow.evaluation.evaluate_prequential import EvaluatePrequential
import numpy as np
import os
from skmultiflow.trees.hoeffding_tree import HoeffdingTreeClassifier
print (inspect.getfile(HoeffdingTreeClassifier))  


from sklearn.metrics import confusion_matrix


/opt/anaconda3/envs/skmultiflow_dev/lib/python3.6/site-packages/scikit_multiflow-0.5.dev0-py3.6-macosx-10.9-x86_64.egg/skmultiflow/trees/hoeffding_tree.py


## Data generation

In [2]:
dataset_feature_names=[     'amount_usd',      
                            'client_age',
                            'client_gender',     # 1 Male other female
                            'debit_type',        # Credit 1or debit 0 card
                            'agency_region',     
                            'merchant_departement',
                            'coicop',
                            'social_class']

#df = pd.read_csv('../data_output/export_dataframe_0v4.csv',header=None,names=dataset_feature_names)
#df = pd.read_csv('../data_output/balanced.csv',header=None,names=dataset_feature_names)
#df.sum()/1000000


## Generate adnostic uniform sample

In [3]:
def generate_uniform_adnostic_sample():
    
    dict_aux = {}
    mu = 0.0 
    sigma = 1.0
    aux = random.uniform(mu, sigma)
    dict_aux['amount_usd']= aux*-1 if (aux<0) else aux
    aux = random.uniform(mu, sigma)
    dict_aux['client_age']= aux*-1 if (aux<0) else aux
    
    aux = random.uniform(mu, sigma)
    dict_aux['client_gender']=1 if (int(round(aux,0)) == 1 ) else 0
    
    aux = random.uniform(mu, sigma)
    dict_aux['debit_type']=1 if (int(round(aux,0)) == 1 ) else 0
        
    aux = random.uniform(mu, sigma)
    dict_aux['agency_region']= int(round(aux * 12,0))
    
    aux = random.uniform(mu, sigma)        
    dict_aux['merchant_departement']= int(round(aux * 25,0))
        
    return dict_aux
    

In [4]:
def generate_gaussian_sample_from_original(pX,py,oracle,minority_label,original=False):
    size = len(py)
    #size = 2000
    print("generate_gaussian_original minority nb: ",size )    
    i = 0
    #mylist = []
    X_aux = []
    
    while (i<size):
        
        row = pX[i].copy()
        #print (type(row))
        i+=1
        if (not original):
            #print (row,i)
            if (np.random.randint(2) == 1):
                mu = 0.006903
                sigma = 0.017028
                aux = random.uniform(mu, sigma)
                row[0] += aux
            elif (np.random.randint(2) == 1):
                mu = 0.006903
                sigma = 0.017028
                aux = random.uniform(mu, sigma)
                row[0] += aux
                mu = 0.343192
                sigma = 0.144008
                aux = random.uniform( sigma,mu)
                row[1] += aux
            else:
                mu = 0.343192
                sigma = 0.144008
                #aux = random.gauss(mu, sigma)
                aux = random.uniform( sigma,mu)
                row[1] += aux
                
        
        y_iter = oracle.predict( np.asarray( [row] ) )

        if (y_iter[0]  == 1):       
            #print ("prediction: ",y_iter,i)
            row = np.append(row,[1],axis=0)
            X_aux.append (row)
    #y_res = oracle.predict( np.asarray(X_aux) )
    #print ("oracle.predict(X) : ", y_res)
    #print (np.unique(y_res,return_counts=True) )
    return pd.DataFrame(X_aux)

In [5]:
def generate_uniform_sample_from_original(pX,py,oracle,minority_label,original=False):
    xmin = np.min(pX,axis=0)
    xman = np.max(pX,axis=0)
    x0_min =  xmin[0]
    x1_min =  xmin[1]
    x0_max =  xman[0]
    x1_max =  xman[1] 
    print (x0_min,x1_min,x0_max,x1_max)
    size = len(py)
    #size = 2000
    print("generate_uniform_from_original minority nb: ",size )    
    i = 0
    #mylist = []
    X_aux = []
    
    while (i<size):
        
        row = pX[i].copy()
        #print (type(row))
        i+=1
        if (not original):
            #print (row,i)
            if (np.random.randint(2) == 1):
                aux = random.uniform(x0_min, x0_max)
                row[0] += aux
            elif (np.random.randint(2) == 1):
                aux = random.uniform(x0_min, x0_max)
                row[0] += aux
                aux = random.uniform( x1_min,x1_max)
                row[1] += aux
            else:
                aux = random.uniform( x1_min,x1_max)
                row[1] += aux
                
        
        y_iter = oracle.predict( np.asarray( [row] ) )
        
        if (y_iter[0]  == 1):       
            #print ("prediction: ",y_iter,i)
            X_aux.append (row)
    #y_res = oracle.predict( np.asarray(X_aux) )
    #print ("oracle.predict(X) : ", y_res)
    #print (np.unique(y_res,return_counts=True) )
    return pd.DataFrame(X_aux)

In [6]:
def generate_syntethic_data (nb_samples,pX,py,oracle,minority_label,pmax_iterations=10):
    
    newdf = pd.DataFrame()
    
    #Minority class
    print ("generating....")
    sampled_df = generate_gaussian_sample_from_original(pX,py,oracle,pmax_iterations)
    nb_sampled = sampled_df.shape[0]
    print("nb_sampled: ",sampled_df.shape)
    
    ii = 0
    print ("while....")
    if (sampled_df.shape[0]<=(nb_samples/2)):
        #print (sampled_df.shape[0]," < ",(nb_samples/2))
        #print ("max_iterations: ",pmax_iterations)
        while   (ii < pmax_iterations):    
            #print ('i : ',ii)
            sampled_df_aux = generate_gaussian_sample_from_original(pX,py,oracle,pmax_iterations)
            sampled_df = sampled_df.append(sampled_df_aux, ignore_index = True) 
            #print (sampled_df_aux.shape)
            #print (sampled_df.shape)
            ii+=1
        if (sampled_df.shape[0]<=(nb_samples/2)):
            ii = 0
            print ("second sampling")
            while   (ii < pmax_iterations  ):
                sampled_df_aux = generate_gaussian_sample_from_original(pX,py,oracle,pmax_iterations,original=True)
                sampled_df = sampled_df.append(sampled_df_aux, ignore_index = False) 
                print (sampled_df.shape)
                ii+=1
                if (sampled_df.shape[0]>=(nb_samples/2)):
                    print ("Break : ", sampled_df.shape)
                    break
    if (sampled_df.shape[0] >= int(nb_samples/2) ):
        newdf = sampled_df.iloc[0:int(nb_samples/2)]#, ignore_index = True)
    else :
        newdf = sampled_df
    #newdf = sampled_df.iloc[0:int(nb_samples/2)]#, ignore_index = True)
    #newdf.append(sampled_df, ignore_index = True)
    
    print ("sampled_df shape", sampled_df.shape)
    print ("sampled_df cols", sampled_df.columns)
    
    #All classes
    
    i = 0
    column_names = ['amount_usd',
                    'client_age',
                    'client_gender',     
                    'debit_type',        
                    'agency_region',     
                    'merchant_departement']
    df = pd.DataFrame(columns = column_names)
    while (i < (nb_samples/2)):
        row ={}
        row = generate_uniform_adnostic_sample()
        df = df.append(row  , ignore_index=True)
        i+=1
        
    #df_client_gender_henc = pd.get_dummies(df['client_gender'], prefix = 'client_gender_')
    #df_debit_type_henc = pd.get_dummies(df['debit_type'], prefix = 'debit_type_')
    #df_agency_region_henc = pd.get_dummies(df['agency_region'], prefix = 'agency_region_')
    #df_merchant_departement_henc = pd.get_dummies(df['merchant_departement'], prefix = 'merchant_departement_')
        
    del df['client_gender']
    del df['debit_type']
    del df['agency_region']
    del df['merchant_departement']
    
    
    newdf2 = pd.concat([df, 
                   df_client_gender_henc,
                   df_debit_type_henc,
                   df_agency_region_henc,
                   df_merchant_departement_henc
                  ], axis=1)
    
    print ("newdf2 shape", newdf2.shape)
    print ("newdf2 cols", newdf2.columns)
    
    newdf2 = newdf2[newdf2['agency_region__12.0'] != 1]
    newdf2 = newdf2[newdf2['merchant_departement__0.0'] != 1]
        
    del newdf2['agency_region__12.0']
    del newdf2['merchant_departement__0.0']
    aX=np.asarray(newdf2)
    aY=oracle.predict(aX)
    newdf2['social_class']=aY
    newdf2.columns = list(range(0,44))
    
    newdf = newdf.append(newdf2)
    print ("newdf", newdf.columns)
    
    newdf = newdf.sample(frac = 1)
    
    return newdf #sampled_df

In [7]:
def compute_y_weight(y):
    aux = np.unique(y,return_counts=True)
    nb_classes = len(aux[0])
    list_classes = aux[0]
    count_classes = aux[1]
    sum_classes =  float(np.sum(count_classes))
    weight_classes = count_classes/sum_classes
    y_weights = y.copy()
    
    i = 0
    while i < nb_classes:
        
        class_i = list_classes[i]
        y_weights = np.where(y_weights==float(class_i), weight_classes[i], y_weights) 
        i += 1
    return y_weights

In [8]:
pCat_features_idx = list(range(2,7))

# Setup the File Stream
stream = FileStream("../data_output/export_dataframe_0v6.csv", -1, 1,  pCat_features_idx)
#stream = FileStream("../data_output/export_dataframe_balanced_v5.csv", -1, 1,  pCat_features_idx)
X, y = stream.next_sample(1000000)
print (np.unique(y,return_counts=True) )
y_weights = compute_y_weight(y)

cfiers = [HoeffdingTreeClassifier(nominal_attributes=pCat_features_idx,split_confidence=1e-12,grace_period=2000,split_criterion='hellinger',binary_split=True), 
          HoeffdingAdaptiveTreeClassifier(nominal_attributes=pCat_features_idx,split_confidence=1e-12,grace_period=2000,split_criterion='hellinger',binary_split=True)
         ]

cfiers[0].partial_fit(X,y,[0,1],y_weights )
cfiers[1].partial_fit(X,y,[0,1],y_weights )
#print(cfiers[0].get_description())


(array([0, 1]), array([851010, 148989]))
HoeffdingTreeClassifier class constructor
HoeffdingTreeClassifier class constructor


HoeffdingAdaptiveTreeClassifier(binary_split=True, bootstrap_sampling=True,
                                grace_period=2000, leaf_prediction='nba',
                                max_byte_size=33554432,
                                memory_estimate_period=1000000, nb_threshold=0,
                                no_preprune=False,
                                nominal_attributes=[2, 3, 4, 5, 6],
                                remove_poor_atts=False, split_confidence=1e-12,
                                split_criterion='hellinger',
                                stop_mem_management=False, tie_threshold=0.05)

In [9]:
y_pred_original_0 = cfiers[0].predict(X)
print("original 0:", np.unique(y,return_counts=True) )
print("prediction 0:", np.unique(y_pred_original_0,return_counts=True) )
print ("precision 0:",np.unique((y==y_pred_original_0),return_counts=True))

#print(cfiers[0].get_model_description())
#(confusion_matrix(y, y_pred_original_0, labels=[0,1]))

tn, fp, fn, tp = confusion_matrix(y, y_pred_original_0).ravel()
print ("tn: {}, fp: {}, fn: {}, tp: {}".format(tn, fp, fn, tp))
print ("precision 0:",tp/(tp+fp))

original 0: (array([0, 1]), array([851010, 148989]))
prediction 0: (array([0, 1]), array([999773,    226]))
precision 0: (array([False,  True]), array([149171, 850828]))
tn: 850806, fp: 204, fn: 148967, tp: 22
precision 0: 0.09734513274336283


In [10]:
y_pred_original_1 = cfiers[1].predict(X)
print("original 1:", np.unique(y,return_counts=True) )
print("prediction 1:", np.unique(y_pred_original_1,return_counts=True) )

#print (confusion_matrix(y, y_pred_original_1, labels=[0,1]))
tn, fp, fn, tp = confusion_matrix(y, y_pred_original_1).ravel()
print ("tn: {}, fp: {}, fn: {}, tp: {}".format(tn, fp, fn, tp))
print ("precision 1:",tp/(tp+fp))
print ("recall 1:",tp/(tp+fn))

original 1: (array([0, 1]), array([851010, 148989]))
prediction 1: (array([0, 1]), array([999542,    457]))
tn: 850555, fp: 455, fn: 148987, tp: 2
precision 1: 0.00437636761487965
recall 1: 1.3423809811462591e-05


In [1]:
np.unique(cfiers[1].predict(X),return_counts=True)

NameError: name 'np' is not defined

## Copy

In [12]:
df = pd.read_csv('../data_output/export_dataframe_0v6.csv',header=None,names=dataset_feature_names)
#df = pd.read_csv('../data_output/export_dataframe_balanced_v5.csv',header=None,names=dataset_feature_names)
oracle = cfiers[1]
column_label = 'social_class'
minority_label = 1
nb_samples = 10000
max_iterations = 30

minority_df = df.loc[df['social_class'] == 1]
#print ("minority",np.unique(minority_df,return_counts=True))
ay=np.asarray(minority_df['social_class'])
aX=np.asarray(minority_df.iloc[:,0:-1])


sampled_df = generate_syntethic_data (nb_samples,X,y,oracle,column_label,max_iterations)

#sampled_df = generate_uniform_adnostic_sample_from_original(X,y,oracle,minority_label,original=True)
#np.unique(cfiers[0].predict(X_synthetic),return_counts=True)
print ("Final ",sampled_df.shape)
sampled_df.head()

generating....
generate_gaussian_original minority nb:  999999
nb_sampled:  (38, 8)
while....
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  999999
generate_gaussian_original minority nb:  9

NameError: name 'df_client_gender_henc' is not defined

In [ ]:
pCat_features_idx = list(range(2,43))


X_copy=np.asarray(sampled_df.iloc[:,0:-1])
y_copy=np.asarray(sampled_df.iloc[:,-1])
y_weights_copy = compute_y_weight(y_copy)

cfiers_copy = [HoeffdingTreeClassifier(nominal_attributes=pCat_features_idx,split_confidence=1e-10, split_criterion='misclassification'), 
          HoeffdingAdaptiveTreeClassifier(nominal_attributes=pCat_features_idx,split_confidence=1e-10,split_criterion='misclassification')
         ]

cfiers_copy[0].partial_fit(X_copy,y_copy,[0,1],y_weights_copy )
cfiers_copy[1].partial_fit(X_copy,y_copy,[0,1],y_weights_copy )


In [ ]:
print(cfiers_copy[0].get_model_description())
print(cfiers_copy[0].measure_byte_size())

In [ ]:
print(cfiers_copy[1].get_model_description())
print(cfiers_copy[1].measure_byte_size())

In [ ]:
print("prediction 1 copy :", np.unique(cfiers_copy[0].predict(X),return_counts=True) )


In [ ]:
print("prediction 2 copy :", np.unique(cfiers_copy[1].predict(X),return_counts=True) )
